In [1]:
import sys
sys.path.append("../")
from utils import *
from instructions import Instruction
from settings import *
from uncompute import *

n = 32
input_file = "../32symbolic_nomem/simple-assignment-1-35.btor2"
current_settings = get_btor2_settings(input_file)
Instruction.all_instructions = read_file(input_file, modify_memory_sort=True, setting=current_settings)
Instruction.with_grover = 0

for i in range(1, n+1):
    print(i)
    Instruction.current_n = i
    for instruction in Instruction.all_instructions.values():
        if instruction[1] == INIT and i == 1:
            Instruction(instruction).execute()
        elif instruction[1] == NEXT or instruction[1] == BAD:
            Instruction(instruction).execute()

result_bad_states = Instruction.or_bad_states()
assert(len(result_bad_states) == 1)

print("stack: ", Instruction.global_stack.size)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
stack:  686939


In [2]:
circuit_queue = get_circuit_queue(Instruction.global_stack)

In [3]:
print(Instruction.circuit.width(), circuit_queue.size)

68240 686939


In [4]:
def are_all_controls_true(values, controls):
    for c in controls:
        if values[c] == 0:
            return False
        else:
            assert(values[c] == 1)
    return True

def check_input(value):
    # we only set the value of the first input the other ones are set to |0>
    qubit_values = {}
    for qword in Instruction.input_nids:
        for i in range(n+1):
            if i in qword.states.keys():
                temp_value = value
                for qubit in qword.states[i][0]:
                    qubit_values[qubit] = temp_value % 2
                    temp_value = temp_value // 2
    element: Element = circuit_queue.pop()
    assert(element.element_type != CHECKPOINT_TYPE)
    while element.element_type != CHECKPOINT_TYPE:
        for o in element.operands:
            if o not in qubit_values.keys():
                qubit_values[o] = 0

        assert (element.target is not None)

        if element.target not in qubit_values.keys():
            qubit_values[element.target] = 0

        flip_target = True
        if element.gate_name == X:
            assert(len(element.operands) == 0)

        else:
            assert((element.gate_name == CX and len(element.operands) ==1) or
                   (element.gate_name == CCX and len(element.operands) == 2) or
                    element.gate_name == MCX)
            flip_target = are_all_controls_true(qubit_values, element.operands)

        if flip_target:
            qubit_values[element.target] = (qubit_values[element.target] + 1) % 2
        circuit_queue.push(element)
        element = circuit_queue.pop()
    assert element.element_type == CHECKPOINT_TYPE
    circuit_queue.push(element)
    lines = []
    for (bad_state, line) in Instruction.bad_states_to_line_no.items():
        if qubit_values[bad_state] == 1:
            lines.append(line)
    return qubit_values[result_bad_states[0]], lines

In [5]:
for i in range(256):
    print(i, check_input(i))

0 (1, [80000013])
1 (1, [80000013])
2 (1, [80000013])
3 (1, [80000013])
4 (1, [80000013])
5 (1, [80000013])
6 (0, [])
7 (0, [])
8 (0, [])
9 (0, [])
10 (0, [])
11 (0, [])
12 (0, [])
13 (0, [])
14 (0, [])
15 (0, [])
16 (0, [])
17 (0, [])
18 (0, [])
19 (0, [])
20 (0, [])
21 (0, [])
22 (0, [])
23 (0, [])
24 (0, [])
25 (0, [])
26 (0, [])
27 (0, [])
28 (0, [])
29 (0, [])
30 (0, [])
31 (0, [])
32 (0, [])
33 (0, [])
34 (0, [])
35 (0, [])
36 (0, [])
37 (0, [])
38 (0, [])
39 (0, [])
40 (0, [])
41 (0, [])
42 (0, [])
43 (0, [])
44 (0, [])
45 (0, [])
46 (0, [])
47 (0, [])
48 (0, [])
49 (1, [80000013])
50 (1, [80000013])
51 (1, [80000013])
52 (1, [80000013])
53 (1, [80000013])
54 (1, [80000013])
55 (1, [80000013])
56 (1, [80000013])
57 (1, [80000013])
58 (1, [80000013])
59 (1, [80000013])
60 (1, [80000013])
61 (1, [80000013])
62 (1, [80000013])
63 (1, [80000013])
64 (1, [80000013])
65 (1, [80000013])
66 (1, [80000013])
67 (1, [80000013])
68 (1, [80000013])
69 (1, [80000013])
70 (1, [80000013])
71 (1